In [1]:
import tensorflow as tf
import sklearn.metrics as sk

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


In [2]:
import matplotlib.pyplot as plt

image_index = 2049 # You may select anything up to 60,000
print(y_train[image_index]) # The label is 5
plt.imshow(x_train[image_index], cmap='Greys')

5


In [3]:
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (60000, 784)
Number of images in x_train 60000
Number of images in x_test 10000


In [4]:
x_train.shape

(60000, 784)

In [5]:
y_train.shape

(60000,)

In [6]:
d = 784 # 28*28 the size of an image
n_hidden = 50
learning_rate = .001
n_epochs = 50
batch_size = 100
dropout_prob = 1.0

with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None,))
    keep_prob = tf.placeholder(tf.float32)
    
with tf.name_scope("hidden-layer"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x, W) + b)
    x_hidden = tf.nn.dropout(x_hidden, keep_prob)
    
with tf.name_scope("output"):
    W = tf.Variable(tf.random_normal((n_hidden, 1)))
    b = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden, W) + b
    # the sigmoid gives the class probability of 1
    y_one_prob = tf.sigmoid(y_logit)
    # Rounding P(y=1) will give the correct prediction.
    y_pred = tf.round(y_one_prob)
    
with tf.name_scope("loss"):
    # Compute the cross-entropy term for each datapoint
    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
    # Sum all contributions
    l = tf.reduce_sum(entropy**2)

with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()
    

In [7]:
x_train.shape[0]

60000

In [8]:
N = x_train.shape[0]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    print(n_epochs)
    for epoch in range(n_epochs):
        pos = 0
        while pos < N:
            batch_X = x_train[pos:pos+batch_size]
            batch_y = y_train[pos:pos+batch_size]
            feed_dict = {x: batch_X, y: batch_y, keep_prob: dropout_prob}
            _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
            if (step % 100 == 0):
                print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
            step += 1
            pos += batch_size
    
    # Make Predictions (set keep_prob to 1.0 for predictions)
    train_y_pred = sess.run(y_pred, feed_dict={x: x_train, keep_prob: 1.0})
    test_y_pred = sess.run(y_pred, feed_dict={x: x_test, keep_prob: 1.0})

    train_weighted_score = sk.accuracy_score(y_train, train_y_pred)
    print("Train Weighted Classification Accuracy: %f" % train_weighted_score)
    test_weighted_score = sk.accuracy_score(y_test, test_y_pred)
    print("Valid Weighted Classification Accuracy: %f" % test_weighted_score)

epoch 0, step 0, loss: 1525008.000000
epoch 0, step 100, loss: 907200.000000
epoch 0, step 200, loss: 376511.812500
epoch 0, step 300, loss: 359591.781250
epoch 0, step 400, loss: 220597.718750
epoch 0, step 500, loss: 348927.625000
epoch 1, step 600, loss: 176054.937500
epoch 1, step 700, loss: 126411.156250
epoch 1, step 800, loss: 94705.656250
epoch 1, step 900, loss: 116547.906250
epoch 1, step 1000, loss: 71465.515625
epoch 1, step 1100, loss: 126313.750000
epoch 2, step 1200, loss: 73727.320312
epoch 2, step 1300, loss: 45896.601562
epoch 2, step 1400, loss: 37905.656250
epoch 2, step 1500, loss: 52692.535156
epoch 2, step 1600, loss: 36448.308594
epoch 2, step 1700, loss: 52007.265625
epoch 3, step 1800, loss: 37791.910156
epoch 3, step 1900, loss: 26987.953125
epoch 3, step 2000, loss: 17772.511719
epoch 3, step 2100, loss: 30749.078125
epoch 3, step 2200, loss: 17714.123047
epoch 3, step 2300, loss: 24753.500000
epoch 4, step 2400, loss: 19804.109375
epoch 4, step 2500, loss: 

epoch 36, step 21600, loss: 10.097156
epoch 36, step 21700, loss: 9.042549
epoch 36, step 21800, loss: 7.704368
epoch 36, step 21900, loss: 10.050305
epoch 36, step 22000, loss: 11.153442
epoch 36, step 22100, loss: 7.989295
epoch 37, step 22200, loss: 9.785727
epoch 37, step 22300, loss: 8.424888
epoch 37, step 22400, loss: 6.964948
epoch 37, step 22500, loss: 9.366642
epoch 37, step 22600, loss: 9.685022
epoch 37, step 22700, loss: 6.365896
epoch 38, step 22800, loss: 8.174784
epoch 38, step 22900, loss: 6.659739
epoch 38, step 23000, loss: 6.391805
epoch 38, step 23100, loss: 7.658178
epoch 38, step 23200, loss: 7.248374
epoch 38, step 23300, loss: 5.199119
epoch 39, step 23400, loss: 6.946386
epoch 39, step 23500, loss: 6.122515
epoch 39, step 23600, loss: 6.032471
epoch 39, step 23700, loss: 7.270615
epoch 39, step 23800, loss: 6.008629
epoch 39, step 23900, loss: 4.662109
epoch 40, step 24000, loss: 6.514796
epoch 40, step 24100, loss: 5.717005
epoch 40, step 24200, loss: 5.68867